# UCI Epileptic Seizure Recognition Data Set

Link to the dataset: https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition

In [12]:
import os
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision import transforms

In [21]:
# path = 'data/'

# dataset = pd.read_csv(path + 'data.csv')

# inputs = dataset.iloc[:, 1:179]
# targets = dataset.iloc[:, 179]

# len(targets)

In [29]:
class UCIEpilepsy(Dataset):
    def __init__(self, data_path, transform=None, target_transform=None):
        self.data = pd.read_csv(data_path)
        self.transform = transform
        self.target_transform = target_transform
        self.data_path = data_path
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        data_path = self.data_path
        data = print(self.data.iloc[idx, :])
        label = self.data.iloc[idx, 178]
        if self.transform:
            data = self.transform(data)
        if self.target_transform: 
            label = self.target_transform(label)
        sample = {"data": data, "label": label}

dataset = UCIEpilepsy(data_path='data/data.csv', transform=transforms.Compose([transforms.ToTensor()]))
batch_size = 64
test_split = .2
shuffle_dataset = True
random_seed = 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, test_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
test_sampler = SubsetRandomSampler(test_indices)

train_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
test_dataloader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

In [30]:
classes = ('Seizure', 'Normal')

In [31]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(178, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 5),
            nn.ReLU()
        )
        
    def forward(self, x):
        return self.linear_relu_stack(x)

In [32]:
device = 'cuda:0'

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=178, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=5, bias=True)
    (5): ReLU()
  )
)


In [33]:
X = torch.rand(1, 178, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [34]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [35]:
for epoch in range(2): # loop over the dataset multiple times
    
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                 (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
            
print('Finished Training')

column_a    X4.V1.911
x1                -66
x2                -25
x3                 12
x4                 45
              ...    
x175               21
x176               34
x177               24
x178                1
y                   4
Name: 6824, Length: 180, dtype: object


TypeError: pic should be PIL Image or ndarray. Got <class 'NoneType'>